In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel 
import numpy as np

In [2]:
IAIM = pd.read_csv('../../../data/trim.csv')
IAIM

,appid,app_title,app_tags,voted_up
0,730,Counter-Strike: Global Offensive,"['FPS', 'Shooter', 'Multiplayer', 'Competitive...",True
1,730,Counter-Strike: Global Offensive,"['FPS', 'Shooter', 'Multiplayer', 'Competitive...",True
2,730,Counter-Strike: Global Offensive,"['FPS', 'Shooter', 'Multiplayer', 'Competitive...",True
3,730,Counter-Strike: Global Offensive,"['FPS', 'Shooter', 'Multiplayer', 'Competitive...",False
4,730,Counter-Strike: Global Offensive,"['FPS', 'Shooter', 'Multiplayer', 'Competitive...",True
...,...,...,...,...
3220600,550,Left 4 Dead 2,"['Zombies', 'Co-op', 'FPS', 'Multiplayer', 'Sh...",True
3220601,1250,Killing Floor,"['FPS', 'Zombies', 'Co-op', 'Survival', 'Horro...",True
3220602,730,Counter-Strike: Global Offensive,"['FPS', 'Shooter', 'Multiplayer', 'Competitive...",True
3220603,730,Counter-Strike: Global Offensive,"['FPS', 'Shooter', 'Multiplayer', 'Competitive...",True


In [4]:
from ast import literal_eval

features = ['app_tags']
for feature in features:
    IAIM[feature] = IAIM[feature].apply(literal_eval)
    
IAIM.head()

,appid,app_title,app_tags,voted_up
0,730,Counter-Strike: Global Offensive,"[FPS, Shooter, Multiplayer, Competitive, Actio...",True
1,730,Counter-Strike: Global Offensive,"[FPS, Shooter, Multiplayer, Competitive, Actio...",True
2,730,Counter-Strike: Global Offensive,"[FPS, Shooter, Multiplayer, Competitive, Actio...",True
3,730,Counter-Strike: Global Offensive,"[FPS, Shooter, Multiplayer, Competitive, Actio...",False
4,730,Counter-Strike: Global Offensive,"[FPS, Shooter, Multiplayer, Competitive, Actio...",True


In [5]:
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]

In [6]:
for feature in features:
    IAIM[feature] = IAIM[feature].apply(clean_data)

In [7]:
IAIM

,appid,app_title,app_tags,voted_up
0,730,Counter-Strike: Global Offensive,"[fps, shooter, multiplayer, competitive, actio...",True
1,730,Counter-Strike: Global Offensive,"[fps, shooter, multiplayer, competitive, actio...",True
2,730,Counter-Strike: Global Offensive,"[fps, shooter, multiplayer, competitive, actio...",True
3,730,Counter-Strike: Global Offensive,"[fps, shooter, multiplayer, competitive, actio...",False
4,730,Counter-Strike: Global Offensive,"[fps, shooter, multiplayer, competitive, actio...",True
...,...,...,...,...
3220600,550,Left 4 Dead 2,"[zombies, co-op, fps, multiplayer, shooter, ac...",True
3220601,1250,Killing Floor,"[fps, zombies, co-op, survival, horror, action...",True
3220602,730,Counter-Strike: Global Offensive,"[fps, shooter, multiplayer, competitive, actio...",True
3220603,730,Counter-Strike: Global Offensive,"[fps, shooter, multiplayer, competitive, actio...",True


In [8]:
def create_usable(x):
    return ' '.join(x['app_tags']) + ' ' + ' '

In [9]:
IAIM['usable'] = IAIM.apply(create_usable, axis=1)

In [10]:
IAIM.head()

,appid,app_title,app_tags,voted_up,usable
0,730,Counter-Strike: Global Offensive,"[fps, shooter, multiplayer, competitive, actio...",True,fps shooter multiplayer competitive action tea...
1,730,Counter-Strike: Global Offensive,"[fps, shooter, multiplayer, competitive, actio...",True,fps shooter multiplayer competitive action tea...
2,730,Counter-Strike: Global Offensive,"[fps, shooter, multiplayer, competitive, actio...",True,fps shooter multiplayer competitive action tea...
3,730,Counter-Strike: Global Offensive,"[fps, shooter, multiplayer, competitive, actio...",False,fps shooter multiplayer competitive action tea...
4,730,Counter-Strike: Global Offensive,"[fps, shooter, multiplayer, competitive, actio...",True,fps shooter multiplayer competitive action tea...


In [12]:
title_and_tags = IAIM.drop(['appid', 'app_tags', 'voted_up'], axis=1)

In [13]:
title_and_tags

,app_title,usable
0,Counter-Strike: Global Offensive,fps shooter multiplayer competitive action tea...
1,Counter-Strike: Global Offensive,fps shooter multiplayer competitive action tea...
2,Counter-Strike: Global Offensive,fps shooter multiplayer competitive action tea...
3,Counter-Strike: Global Offensive,fps shooter multiplayer competitive action tea...
4,Counter-Strike: Global Offensive,fps shooter multiplayer competitive action tea...
...,...,...
3220600,Left 4 Dead 2,zombies co-op fps multiplayer shooter action o...
3220601,Killing Floor,fps zombies co-op survival horror action multi...
3220602,Counter-Strike: Global Offensive,fps shooter multiplayer competitive action tea...
3220603,Counter-Strike: Global Offensive,fps shooter multiplayer competitive action tea...


In [14]:
title_and_tags = title_and_tags.drop_duplicates()
title_and_tags['app_title'].unique()

array(['Counter-Strike: Global Offensive', 'Left 4 Dead 2', 'Portal 2',
       'The Ship: Murder Party', 'Day of Defeat: Source', 'Half-Life',
       'Half-Life: Opposing Force', 'Portal', 'Team Fortress 2',
       'Counter-Strike', 'Counter-Strike: Source', 'Left 4 Dead',
       'Killing Floor', 'Day of Defeat', 'Team Fortress Classic',
       'Wolfenstein 3D', 'Half-Life 2', 'Counter-Strike: Condition Zero',
       'Half-Life 2: Episode Two', 'Call of Duty® 2', 'Sniper Elite',
       'DOOM II', 'Half-Life 2: Deathmatch', 'Ultimate Doom', 'Ricochet',
       'Half-Life: Source', 'Quake III Arena', 'Half-Life 2: Lost Coast',
       'Half-Life: Blue Shift', 'Judge Dredd: Dredd vs. Death',
       'Alpha Prime', 'Half-Life 2: Episode One', 'QUAKE III: Team Arena',
       'QUAKE', 'Aliens versus Predator Classic 2000',
       'Deathmatch Classic', 'QUAKE II', 'Call of Juarez',
       'Bloody Good Time', 'Half-Life Deathmatch: Source',
       'Call of Duty: United Offensive', 'Red Orchestra:

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(title_and_tags['usable'])

In [16]:
count_matrix.shape

(175, 205)

In [17]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [18]:
recommend = title_and_tags.reset_index()
indices = pd.Series(recommend.index, index=recommend['app_title'])

In [25]:
def get_recommendations(title, score, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    if score == 0:
        sim_scores = sim_scores[165:176]
    elif score == 1:
        sim_scores = sim_scores[1:11]

    # Get the movie indices
    game_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return recommend['app_title'].iloc[game_indices]

In [26]:
get_recommendations('Half-Life', 1, cosine_sim)

6                            Half-Life: Opposing Force
25                                   Half-Life: Source
28                               Half-Life: Blue Shift
125    STAR WARS™ Jedi Knight - Mysteries of the Sith™
16                                         Half-Life 2
42                                            Quake IV
33                                               QUAKE
139                           STAR WARS™ - Dark Forces
31                            Half-Life 2: Episode One
113                                             Crysis
Name: app_title, dtype: object

In [27]:
get_recommendations('Half-Life', 0, cosine_sim)

160                         Rogue Warrior
136                           Men of War™
49                ARMA: Combat Operations
151                      Revelations 2012
146         Space Trader: Merchant Marine
107                   Men of War: Vietnam
124             Men of War: Assault Squad
148            Hamilton's Great Adventure
111    Brothers in Arms: Earned in Blood™
8                         Team Fortress 2
Name: app_title, dtype: object